# Web scraping with Python

This notebook demonstrates how you can use the Python programming language to scrape information from a web page. The goal today: Scrape the main table on [the first page of Maryland's list of WARN letters](https://www.dllr.state.md.us/employment/warn.shtml) and, if time, write the data to a CSV.

If you're relatively new to Python, it might be helpful to have [this Python syntax cheat sheet](Python%20syntax%20cheat%20sheet.ipynb) open in another tab as you work through this notebook.

### Table of contents

- [Using Jupyter notebooks](#Using-Jupyter-notebooks)
- [What _is_ a web page, anyway?](#What-is-a-web-page,-anyway?)
- [Inspect the source](#Inspect-the-source)
- [Import libraries](#Import-libraries)
- [Request the page](#Request-the-page)
- [Turn your HTML into soup](#Turn-your-HTML-into-soup)
- [Targeting and extracting data](#Targeting-and-extracting-data)
- [Write the results to file](#Write-the-results-to-file)

### Using Jupyter notebooks

There are several ways to write and run Python code on your computer. One way -- the method we're using today -- is to use [Jupyter notebooks](https://jupyter.org/), which run in your browser and allow you to intersperse documentation with your code. They're handy for bundling your code with a human-readable explanation of what's happening at each step. Check out some examples from the [L.A. Times](https://github.com/datadesk/notebooks) and [BuzzFeed News](https://github.com/BuzzFeedNews/everything#data-and-analyses).

**To add a new cell to your notebook**: Click the + button in the menu or press the `b` button on your keyboard.

**To run a cell of code**: Select the cell and click the "Run" button in the menu, or you can press Shift+Enter.

**One common gotcha**: The notebook doesn't "know" about code you've written until you've _run_ the cell containing it. For example, if you define a variable called `my_name` in one cell, and later, when you try to access that variable in another cell but get an error that says `NameError: name 'my_name' is not defined`, the most likely solution is to run (or re-run) the cell in which you defined `my_name`.

### What _is_ a web page, anyway?

Generally, a web page consists of a bunch of specifically formatted text files stored on a computer (a _server_) that's probably sitting on a rack in a giant data center somewhere.

Mostly you'll be dealing with `.html` (HyperText Markup Language) files that might include references to `.css` (Cascading Style Sheet) files, which determine how the page looks, and/or `.js` (JavaScript) files, which add interactivity, and other specially formatted text files.

Today, we'll focus on the HTML, which gives structure to the page.

Most HTML elements are represented by a pair of tags -- an opening tag and a closing tag.

A table, for example, starts with `<table>` and ends with `</table>`. The first tag tells the browser: "Hey! I got a table here! Render it as a table." The closing tag (note the forward slash!) tells the browser: "Hey! I'm all done with that table, thanks." Inside the table are nested more HTML tags representing rows (`<tr>`) and cells (`<td>`).

HTML elements can have any number of attributes, such as classes --

`<table class="cool-table">`

-- styles --

`<table style="width:95%;">`

-- hyperlinks to other pages --

`<a href="https://ire.org">Click here to visit IRE's website</a>`

-- and IDs --

`<table id="cool-table">`

-- that will be useful to know about when we're scraping.

### Inspect the source

You can look at the HTML that makes up a web page by _inspecting the source_ in a web browser. We like Chrome and Firefox for this; today, we'll use Chrome.

You can inspect specific elements on the page by right-clicking on the page and selecting "Inspect" or "Inspect Element" from the context menu that pops up. Hover over elements in the "Elements" tab to highlight them on the page.

To examine all of the source code that makes up a page, you can "view source." In Chrome, hit `Ctrl+U` on a PC or `⌘+Opt+U` on a Mac. (It's also in the menu bar: View > Developer > View Page Source.)

You'll get a page showing you all of the HTML code that makes up that page. Ignore 99% of it and try to locate the element(s) that you want to target (use `Ctrl+F` on a PC and `⌘+F` to find).

Open up a Chrome browser and inspect the table on the [the first page of Maryland's list of WARN letters](https://www.dllr.state.md.us/employment/warn.shtml). Find the table we want to scrape.

Is it the only table on the page? If not, does it have any attributes that would allow you to target it?

### Import libraries

Step one is to _import_ two third-party Python libraries that will help us scrape this page:
- `requests` is the de facto standard for making HTTP requests, similar to what happens when you type a URL into a browser window and hit enter.
- `bs4`, or BeautifulSoup, is a popular library for parsing HTML into a data structure that Python can work with.

These libraries are installed separately from Python on a per-project basis ([read more about our recommendations for setting up Python projects here](https://docs.google.com/document/d/1cYmpfZEZ8r-09Q6Go917cKVcQk_d0P61gm0q8DAdIdg/edit#heading=h.od2v1nkge5t1)).

Run this cell (you'll only have to do this once):

In [ ]:
import requests
import bs4

### Request the page

Next, we'll use the `get()` method of the `requests` library (which we just imported) to grab the web page.

While we're at it, we'll _assign_ all the stuff that comes back to a new variable using `=`.

The variable name is arbitrary, but it's usually good to pick something that describes the value it's pointing to.

Notice that the URL we're grabbing is wrapped in quotes, making it a _string_ that Python will interepret as text (as opposed to numbers, booleans, etc.). You can read up more on Python data types and variable assignment [here](Python%20syntax%20cheat%20sheet.ipynb).

Run these two cells:

In [ ]:
URL = 'http://www.dllr.state.md.us/employment/warn.shtml'

In [ ]:
warn_page = requests.get(URL)

Nothing appears to have happened, which is (usually) a good sign.

If you want to make sure that your request was successful, you can check the `status_code` attribute of the Python object that was returned:

In [ ]:
warn_page.status_code

A `200` code means all is well. `404` means the page wasn't found, etc. ([Here's one of our favorite lists of HTTP status codes](https://http.cat/) ([or here, if you prefer dogs](https://httpstatusdogs.com/)).)

The object being stored as the `warn_page` variable came back with a lot of potentially useful information we could access. Today, we're mostly interested in the `.text` attribute -- the HTML that makes up the web page, same as if we'd viewed the page source. Let's take a look:

In [ ]:
warn_page.text

### ✍️ Try it yourself

Use the code blocks below to experiment with requesting web pages and checking out the HTML that gets returned.

Some ideas to get you started:
- `'http://ire.org'`
- `'https://web.archive.org/web/20031202214318/http://www.tdcj.state.tx.us:80/stat/finalmeals.htm'`
- `'https://www.nrc.gov/reactors/operating/list-power-reactor-units.html'`

### Turn your HTML into soup

The HTML in the `.text` attribute of the request object is just a string -- a big ol' chunk of text.

Before we start targeting and extracting pieces of data in the HTML, we need to turn that chunk of text into a data structure that Python can work with. That's where the [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) (`bs4`) library comes in.

We'll create a new instance of a `BeautifulSoup` object, which lives under the top-level `bs4` library that we imported earlier. We need to give it two things:
- The HTML we'd like to parse -- `warn_page.text`
- A string with the name of the type of parser to use -- `html.parser` is the default and usually fine, but [there are other options](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#installing-a-parser)

We'll save the parsed HTML as a new variable, `soup`.

In [ ]:
soup = bs4.BeautifulSoup(warn_page.text, 'html.parser')

Nothing happened, which is good! You can take a look at what `soup` is, but it looks pretty much like `warn_page.text`:

In [ ]:
soup

If you want to be sure, you can use the Python function `type()` to check what sort of object you're dealing with:

In [ ]:
# the `str` type means a string, or text
type(warn_page.text)

In [ ]:
# the `bs4.BeautifulSoup` type means we successfully created the object
type(soup)

### ✍️ Try it yourself

Use the code blocks below to experiment fetching HTML and turning it into soup (if you fetched some pages earlier and saved them as variables, that'd be a good start).

### Targeting and extracting data

Now that we have BeautifulSoup object loaded up, we can go hunting for the specific HTML elements that contain the data we need. Our general strategy:
1. Find the main table with the data we want to grab
2. Get a list of rows (the `tr` element, which stands for "table row") in that table
3. Use a Python `for loop` to go through each table row and find the data inside it (`td`, or "table data")

To accomplish this, we'll use two `bs4` methods:
- [`find()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find), which returns the first element that matches whatever criteria you hand it
- [`find_all()`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all), which returns a _list_ of elements that match the criteria. ([Here's how Python lists work](Python%20syntax%20cheat%20sheet.ipynb#Lists).)

#### Find the table

To start with, we need to find the table. There are several ways to accomplish this, but because this is the only table on the page (view source and `Ctrl+F` to search for `<table` to confirm), we can simply say, "Look through the `soup` object and find the table tag."

Translated, the code is: `soup.find('table')`. While we're at it, save the results of that search to a new variable, `table`.

Run these cells:

In [ ]:
table = soup.find('table')

In [ ]:
table

#### Find the rows in the table

Next, use the `find_all()` method to drill down and get a list of rows in the table:

In [ ]:
rows = table.find_all('tr')

In [ ]:
rows

To see how many items are in this list -- in other words, how many rows are in the table -- you can use the `len()` function:

In [ ]:
len(rows)

#### Loop through the rows and extract the data

Next, we can use a [`for` loop](Python%20syntax%20cheat%20sheet.ipynb#for-loops) to go through the list of rows and start grabbing data from each one.

Quick refresher on _for loop_ syntax: Start with the word `for` (lowercase), then a variable name to stand in for each item in the list that you're looping over, then the word `in` (lowercase), then the name of the list holding the items (`rows`, in our case), then a colon, then an indented block of code describing what we're doing to each item in the list.

Each piece of data in the row will be stored in a `td` tag, which stands for "table data." So inside the loop -- in the indented block -- we'll use the `find_all()` method to get a list of every `td` tag inside the row. And from there, we can access the content inside each tag.

Our goal is to end up with a _list_ of data for each row that we will eventually write out to a file. Typically you'd probably do the work of looping and inspecting the results, step by step, in one code cell. But to show the thinking of how you might approach this (and to practice the syntax), we'll start by just printing out each row and then build from there. (`print('')` will print a blank line to help us see exactly what we're working with in each row.)

In [ ]:
for row in rows:
    print(row)
    print('')

Notice that the first item that prints is the header row with the column labels. You are free to keep these headers if you want, but I typically skip that row and define my own list of column names.

(Another thing to consider: On better-constructed web pages, the cells in the header row will be represented by `th` ("table header") tags, not `td` ("table data") tags. The next step in our `for` loop is, "Find all of the `td` tags in this row," so that would be something you would need to deal with.)

We can skip the first row by using _list slicing_: adding square brackets after the name of the list with some instructions about which items in the list we want to select.

Here, the syntax would be: `rows[1:]`, which means, take everything in the `rows` list starting with the item in position 1 (the second item) to the end of the list. Like many programming languages, Python starts counting at 0, so the result will leave off the first item in the list -- i.e. the item in position 0, i.e. the headers.

In [ ]:
for row in rows[1:]:
    print(row)
    print('')

Now we're cooking with gas. Let's start pulling out the data in each row. Start by using `find_all()` to grab a list of `td` tags:

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    print(cells)
    print('')

Now we have, for each row, a _list_ of `td` tags. Next step is to look at the table and start grabbing specific values based on their position in the list and assigning them to human-readable variable names.

Quick refresher on list syntax: To access a specific item in a list, use square brackets `[]` and the index number of the item you'd like to access. For instance, to get the first cell in the row -- the date that each WARN report was issued -- use `[0]`.

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0]
    print(warn_date)
    print('')

This is returning the entire `Tag` object -- we just want the contents inside it. You can access the `.text` attribute of the tag to get the text inside:

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text    
    print(warn_date)

In the next cell (`[1]`), the `.text` attribute will give you the NAICS code. In the third cell (`[2]`) you'll get the name of the business. Etc.

It's also generally good practice to trim off external whitespace for each value, and you can use the Python built-in string method `strip()` to accomplish this as you march across the row.

Which gets us this far:

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text.strip()
    naics_code = cells[1].text.strip()
    biz = cells[2].text.strip()
    print(warn_date, naics_code, biz)

### ✍️ Try it yourself

Now that you've gotten this far, see if you can isolate the other pieces of data in each row.

In [ ]:
for row in rows[1:]:
    cells = row.find_all('td')
    warn_date = cells[0].text.strip()
    naics_code = cells[1].text.strip()
    biz = cells[2].text.strip()
    
    # address
    
    # wia_code
    
    # total_employees
    
    # effective_date
    
    # type_code

    # print()

### Write the results to file

Now that we've targeted our lists of data for each row, we can use Python's built-in [`csv`](https://docs.python.org/3/library/csv.html) module to write each list to a CSV file.

First, import the csv module. (Typically, the convention is to handle all of your import statements at the top of your script, but it's no big deal if you do it here.)

In [ ]:
import csv

Now define a list of headers to match the data (each column header will be a string) -- run this cell:

In [ ]:
HEADERS = [
    'warn_date',
    'naics_code',
    'biz',
    'address',
    'wia_code',
    'total_employees',
    'effective_date',
    'type_code'
]

Now, using something called a `with` block, open a new CSV file to write to and write some code to do the following things:
- Create a `csv.writer` object
- Write out the list of headers using the `writerow()` method of the `csv.writer` object
- Drop in the `for` loop you just wrote and, instead of just printing the contents of each cell, create a list of items and use the `writerow()` method of the `csv.writer` object to write your list of data to file

In [ ]:
# create a file called 'warn-data.csv' in write ('w') mode
# specify that newlines are terminated by an empty string (this deals with a PC-specific problem)
# and use the `as` keyword to name the open file handler (the variable name `outfile` is arbitrary)
with open('warn-data.csv', 'w', newline='') as outfile:
    # go to the csv module we imported and make a new .writer object attached to the open file
    # and save it to a variable
    writer = csv.writer(outfile)

    # write out the list of headers
    writer.writerow(HEADERS)
    
    # paste in the for loop you wrote earlier here -- watch the indentation!
    # it should be at this indentation level =>
    # for row in rows[1:]:
    #     cells = row.find_all('td')
    #     etc. ...
    # but at the end, instead of `print(warn_date, naics_code, ...etc.)`
    # make it something like
    # data_out = [warn_date, naics_code, ...etc.]
    # `writer.writerow(data_out)`

If you look in the folder, you should see a new file: `warn-data.csv`. Hooray!

🎉 🎉 🎉

### ✍️ Try it yourself

Putting it all together:
- Find a website you'd like to scrape
- Use `requests` to fetch the HTML
- Use `bs4` to parse the HTML and isolate the data you're interested in
- Use `csv` to write the data to file

### Extra credit problems

1. **Remove internal whitespace:** Looking over the data, you probably noticed that some of the values have some unnecessary internal whitespace, which you could fix before you wrote each row to file. Python does not have a built-in string method to remove internal whitespace, unfortunately, but [Googling around](https://www.google.com/search?q=python+remove+internal+whitespace) will yield you a common strategy: Using the `split()` method to separate individual words in the string, then `join()`ing the resulting list on a single space. As an example:

```python
my_text = 'hello     world      how are      you?'

# split() will turn this into a list of words
my_text_words = my_text.split()
# ['hello', 'world', 'how', 'are', 'you?']

# join on a single space
my_text_clean = ' '.join(my_text_words)
print(my_text_clean)
# prints 'hello world how are you?'

# or, as a one-liner
my_text_clean = ' '.join(my_text.split())
```

2. **Fetch multiple years:** The table we scraped has WARN notices for the current year, but the agency also maintains pages with WARN notices for previous years -- there's a list of them in a section [toward the bottom of the page](https://www.dllr.state.md.us/employment/warn.shtml). See if you can figure out how to loop over multiple pages and scrape the contents of each into a single CSV.


3. **Build a lookup table:** Each numeric code in the "WIA Code" column correspondes to a local area. See if you can figure out how to create a lookup dictionary that maps the numbers to their locations, then as you're looping over the data table, replace the numeric value in that column with the name of the local area instead. Here's a hint:

```python
    lookup_dict = {
        '1': 'hello',
        '2': 'world'
    }

    print(lookup_dict.get('1'))
    # prints 'hello'

    print(lookup_dict.get('3'))
    # prints None

```


4. **Fix encoding errors:** You might have noticed a few encoding problems -- e.g., `Nestlé` is being renedered as `NestlÃ©`. This is due to an encoding problem -- the `warn_page.text` is not encoded as `utf-8`. Using `decode()` and `encode()`, see if you can fix this. (Hint! It looks like the state of Maryland is a big fan of `latin-1`.)